# Intro

[Kivy](https://kivy.org/#home) is a python library to make cross platform applications. Kivy apps can handle touch events like pressing a button.

This post shows how to create a custom [Button](https://kivy.org/doc/stable/api-kivy.uix.button.html) that can handle single and double tap events. 

# Implementation

Kivy Buttons receive a [```TouchEvents```](https://kivy.org/doc/stable/guide/inputs.html#touch-event-basics) whenever it is touched. ```TouchEvents``` have a [```is_double_tap```](https://kivy.org/doc/stable/api-kivy.input.html?highlight=is_double_tap#kivy.input.MotionEvent.is_double_tap) property to tell if the event was a double tap. However, there is no native way to distinguish a single tap from a double tap. We need to create a custom button to handle single and double tap events.

We create a sublcass of ```Button``` called ```MultiTouchButton```. The ```on_single_tap``` function executes on single tap events and ```on_double_tap``` function executes on double tap events. You will need to subclass the ```MultiTouchButton``` and complete the function definitions for ```on_single_tap``` and ```on_double_tap``` based on your app logic.

Kivy widgets execute [```on_touch_down```](https://kivy.org/doc/stable/api-kivy.uix.widget.html#kivy.uix.widget.Widget.on_touch_down) when it receives a touch down event. We need to write custom code in the ```on_touch_down``` handler that can distinguish between single and double tap events. 

The ```touch``` parameter to ```on_touch_down``` is a ```TouchEvent```, so we can easily call ```is_double_tap``` to confirm if it is a double tap event. We then call ```self.on_double_tap``` to execute the app specific logic.

If the event is not a double tap event then schedule to run ```self.on_single_tap``` if the widget does not receive any new touch down event in the next ```double_tap_wait_s``` seconds. If the widget receives a new touch down event then the scheduled call to ```self.on_single_tap``` is canceled.


In [ ]:
from kivy.uix.button import Button
from kivy.clock import Clock


class MultiTapButton(Button):

    scheduled_event = None
    
    def on_single_tap(self, *args):
        pass

    def on_double_tap(self, *args):
        pass

    # on_touch_down is a touch event handler for all Kivy widgets
    def on_touch_down(self, touch):

        # By default, touch events are dispatched to all currently displayed widgets. 
        # This means widgets receive the touch event whether it occurs within their physical area or not.
        # Call self.collide_point() to ensure widget responds to touch events on the widget
        if self.collide_point(*touch.pos):

            if self.scheduled_event is not None:
                self.scheduled_event.cancel()
                self.scheduled_event = None
            
            if touch.is_double_tap:
                self.on_double_tap()
            else:
                double_tap_wait_s = 1

                self.scheduled_event = Clock.schedule_once(
                    self.on_single_tap, double_tap_wait_s
                )

# Example

Now subclass the ```MultiTapButton``` and implement the ```on_single_tap``` and ```on_double_tap``` functions.

Below we create a button that prints "single tap" after a single tap event and prints "double tap" after a double tap event

In [ ]:
class TestButton(MultiTapButton):
    def on_single_tap(self, *args):
        print("single tap")

    def on_double_tap(self, *args):
        print("double tap")

---

Resources

- [Solution inspiration](https://stackoverflow.com/questions/64741710/python-3-kivy-react-only-to-double-tap-not-single-tap#)
- [Kivy Touch events](https://kivy.org/doc/stable/guide/inputs.html#touch-event-basics)